In [1]:
# repo url(https://github.com/xwang71785/buffett.git)
# 只针对中国大陆股票市场
# 仅限于概念验证，勿用于实际投资中
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
import pandas_datareader.data as web
import datetime as dt
import matplotlib.dates as mdates

In [3]:
# 基于网易构造三张财务报表的数据源URL，根据需求变更stock中的股票代码
server = "http://quotes.money.163.com/"
path = "service/"
table1 = "zcfzb_"
table2 = "lrb_"
table3 = "xjllb_"
stock = "600585"
type = ".html?type=year"

target1 = server + path + table1 + stock + type
target2 = server + path + table2 + stock + type
target3 = server + path + table3 + stock + type

print(target1, '\n', target2, '\n', target3)

http://quotes.money.163.com/service/zcfzb_600585.html?type=year 
 http://quotes.money.163.com/service/lrb_600585.html?type=year 
 http://quotes.money.163.com/service/xjllb_600585.html?type=year


In [4]:
# 获取三张财务报表CSV格式原始数据
df1 = pd.read_csv(target1, header=0, prefix="V", encoding="gb18030")
df2 = pd.read_csv(target2, header=0, prefix="V", encoding="gb18030")
df3 = pd.read_csv(target3, header=0, prefix="V", encoding="gb18030")

In [5]:
# 报表行列转置
bs = df1.transpose()
pl = pd.DataFrame(df2.values.T, columns=df2['报告日期'], index=df2.columns)
cf = pd.DataFrame(df3.values.T, columns=df3[' 报告日期'], index=df2.columns)

In [6]:
bs.drop('报告日期', inplace=True)    # 删除第一行
bs.drop(bs.tail(1).index, inplace=True)    # 删除最后一行

In [7]:
# 将字符型数据转换成数值型
headers_bs = bs.columns
headers_pl = pl.columns
headers_cf = cf.columns


for head in headers_bs:
    bs[head] = pd.to_numeric(bs[head], errors='coerce').fillna(0) # 不符合转换条件的设为‘Nan’，再用0替换

for head in headers_pl:
    pl[head] = pd.to_numeric(pl[head], errors='coerce').fillna(0)

for head in headers_cf:
    cf[head] = pd.to_numeric(cf[head], errors='coerce').fillna(0)

In [8]:
# 整理资产负债表
bs_sum = pd.DataFrame()    # 利用column的数字标识进行分类汇总

bs_sum['operat_current'] = bs[0]+bs[6]+bs[7]+bs[8]+bs[12]+bs[13]+bs[14]+bs[15]+bs[17]+bs[20]+bs[21]+bs[22]+bs[23]+bs[19]
bs_sum['operat_non_curr'] = bs[29]+bs[28]+bs[33]+bs[34]+bs[35]+bs[36]+bs[37]+bs[38]+bs[39]+bs[40]+bs[32]+bs[43]+bs[44]+bs[45]+bs[46]+bs[42]+bs[49]+bs[48]
bs_sum['operat_liability'] = bs[57]+bs[60]+bs[59]+bs[64]+bs[63]+bs[67]+bs[66]+bs[70]+bs[71]+bs[72]+bs[69]+bs[78]+bs[79]+bs[80]+bs[77]+bs[82]+bs[87]+bs[91]+bs[90]
bs_sum['financial_assets'] = bs[2]+bs[3]+bs[4]+bs[5]+bs[1]+bs[10]+bs[11]+bs[9]+bs[16]+bs[18]+bs[26]+bs[27]+bs[25]+bs[31]+bs[30]+bs[41]+bs[47]
bs_sum['financial_liability'] = bs[53]+bs[54]+bs[55]+bs[56]+bs[52]+bs[58]+bs[62]+bs[61]+bs[65]+bs[68]+bs[74]+bs[75]+bs[76]+bs[73]+bs[81]+bs[85]+bs[86]+bs[84]+bs[89]+bs[88]
bs_sum['equity'] = bs[95]+bs[94]+bs[98]+bs[97]+bs[99]+bs[101]+bs[103]+bs[105]
# 计算资本支出
bs_sum['operat_net_assets'] = bs_sum['operat_current'] + bs_sum['operat_non_curr'] - bs_sum['operat_liability']
bs_sum['capital_expense'] = bs_sum['operat_net_assets'] - bs_sum['operat_net_assets'].shift(-1)

bs_sum = bs_sum.head(5)   # 只取过去5年数据

total_shares = bs[94]    # 从资产负债表的‘实收资本’中获取流通股数量。疑问:数据中标明是金额万元，但核对后确认是流通股数量

In [9]:
bs_sum 

,operat_current,operat_non_curr,operat_liability,financial_assets,financial_liability,equity,operat_net_assets,capital_expense
2021-12-31,8709450.0,34280458.0,2252210.0,3233668.0,1286922.0,19190169.0,40737698.0,3379806.0
2020-12-31,7447513.0,31199939.0,1289560.0,3356807.0,1098629.0,16819758.0,37357892.0,3430156.0
2019-12-31,6801131.0,29149102.0,2022497.0,2582363.0,1224511.0,14217694.0,33927736.0,2125349.0
2018-12-31,5853693.0,27861927.0,1913233.0,1351835.0,1068405.0,11630762.0,31802387.0,3237326.0
2017-12-31,3466429.0,26672810.0,1574178.0,1178644.0,1443670.0,9167292.0,28565061.0,1106694.0


In [12]:
# 整理利润表
pl_sum = pd.DataFrame()

pl_sum['total_revenu'] = pl['营业总收入(万元)']
pl_sum['growth_rate'] = (pl['营业总收入(万元)'] - pl['营业总收入(万元)'].shift(-1)) / pl['营业总收入(万元)'].shift(-1)     # 计算历史每年销售增长率
pl_sum['intre_expen'] = pl['利息支出(万元)']
pl_sum['total_profit'] = pl['利润总额(万元)']
pl_sum['net_profit'] = pl['净利润(万元)']
pl_sum['income_tax'] = pl['所得税费用(万元)']
pl_sum['tax_rate'] = pl['所得税费用(万元)'] / pl['利润总额(万元)']    # 计算企业所得税率

pl_sum.drop('报告日期', inplace=True)    # 删除第一行
pl_sum = pl_sum.head(5)    # 提取历史5年数据

In [13]:
pl_sum

,total_revenu,growth_rate,intre_expen,total_profit,net_profit,income_tax,tax_rate
2021-12-31,16795266.0,-0.047038,0.0,4411620.0,3416584.0,995036.0,0.225549
2020-12-31,17624268.0,0.122348,0.0,4710792.0,3637018.0,1073774.0,0.227939
2019-12-31,15703033.0,0.222953,0.0,4455685.0,3435201.0,1020484.0,0.229030
2018-12-31,12840263.0,0.704969,0.0,3962920.0,3063601.0,899318.0,0.226933
2017-12-31,7531082.0,0.346473,0.0,2122876.0,1642873.0,480002.0,0.226109


In [14]:
# 整理现金流量表
cf_sum = pd.DataFrame()
# 提取和资本支出相关项目数据
cf_sum['depr_fixed'] = cf[' 固定资产折旧、油气资产折耗、生产性物资折旧(万元)']    # 注意column的标题有一个前置空格
cf_sum['intang_amort'] = cf[' 无形资产摊销(万元)']
cf_sum['deferred'] = cf[' 长期待摊费用摊销(万元)']
# cf_sum['depreciation'] = cf[' 固定资产折旧、油气资产折耗、生产性物资折旧(万元)']+cf[' 无形资产摊销(万元)']+cf[' 长期待摊费用摊销(万元)']

cf_sum.drop('报告日期', inplace=True)    # 删除第一行
cf_sum = cf_sum.head(5)

In [15]:
cf_sum

,depr_fixed,intang_amort,deferred
2021-12-31,535191.0,60485.0,0.0
2020-12-31,498444.0,44512.0,0.0
2019-12-31,493163.0,33934.0,0.0
2018-12-31,464730.0,30611.0,0.0
2017-12-31,459028.0,24290.0,0.0


In [16]:
# 汇总历史估值数据
valuation = pd.DataFrame()

valuation['profit_after_tax'] = pl_sum['net_profit'] + pl_sum['intre_expen'] * (1 - pl_sum['tax_rate'] )
valuation['depreciation'] = cf_sum['depr_fixed'] + cf_sum['intang_amort'] + cf_sum['deferred']
valuation['capital_expense'] = bs_sum['capital_expense']
valuation['net_cash_flow'] = valuation['profit_after_tax'] + valuation['depreciation'] - valuation['capital_expense']

In [17]:
valuation

,profit_after_tax,depreciation,capital_expense,net_cash_flow
2021-12-31,3416584.0,595676.0,3379806.0,632454.0
2020-12-31,3637018.0,542956.0,3430156.0,749818.0
2019-12-31,3435201.0,527097.0,2125349.0,1836949.0
2018-12-31,3063601.0,495341.0,3237326.0,321616.0
2017-12-31,1642873.0,483318.0,1106694.0,1019497.0


In [85]:
# 估算收入的增长率
growth_rate_max = 1+ pl_sum['growth_rate'].max()    # 最大
growth_rate_mean = 1 + pl_sum['growth_rate'].mean()    # 平均
growth_rate_min = 1 + pl_sum['growth_rate'].min()    # 最小
# growth_rate3 = np.percentile(pl_sum['growth_rate'], 75)    # 75分位

growth_rate = growth_rate_mean
growth_rate_continue = 1.05


# 未来5年的增长比例，第6个是永续经营的比例
a = np.logspace(1,5,5, base=growth_rate)
growth_rates = np.append(a, a[4] * growth_rate_continue)

discount_rate = 0.12    # 折现率，仅供参考
continue_rate = 0.05

In [50]:
growth_rates
revenu_forecast = pl_sum.iloc[0, 0] * growth_rates
revenu_forecast

array([21328999.55153494, 27086574.38765053, 34398355.64181097,
       43683887.59414264, 55475966.79354594, 58249765.13322324])

In [41]:
pl_sum['total_revenu']

2021-12-31    16795266.0
2020-12-31    17624268.0
2019-12-31    15703033.0
2018-12-31    12840263.0
2017-12-31     7531082.0
Name: total_revenu, dtype: float64

In [45]:
# 预测未来净利润
profit_forecast = valuation.iloc[0,0] * growth_rates


In [47]:
profit_forecast

array([ 4338860.64107478,  5510097.70655949,  6997499.86169443,
        8886413.07687215, 11285221.7125564 , 11849482.79818423])

In [40]:
# 预测未来5年资本支出
bs_rate = bs_sum.sum() / pl_sum['total_revenu'].sum()    # 各资本项占销售收入的历史比率
bs_rate

operat_current         0.457887
operat_non_curr        2.115987
operat_liability       0.128404
financial_assets       0.166019
financial_liability    0.086846
equity                 1.007543
operat_net_assets      2.445470
capital_expense        0.188376
dtype: float64

In [52]:
bs_forecast = bs_rate.head(5) * revenu_forecast[0]    # 当年bs项的预测，只提取前5项

# b = np.logspace(0,4,5, base=growth_rate_max)
# np.append(b, b[4] * 1.05)

In [53]:
#np.expand_dims(b, 0).repeat(8, axis=0)
#np.expand_dims(bs_forecast.T, 0).repeat(5, axis=1)
bs_last = bs_sum.iloc[0].head(5)

In [54]:
bs_forecast1 = bs_forecast * growth_rate
bs_forecast2 = bs_forecast1 * growth_rate
bs_forecast3 = bs_forecast2 * growth_rate
bs_forecast4 = bs_forecast3 * growth_rate
bs_forecast5 = bs_forecast4 * growth_rate_continue

In [55]:
operate = pd.concat([bs_last, bs_forecast, bs_forecast1, bs_forecast2, bs_forecast3, bs_forecast4, bs_forecast5], axis=1)
operate

,2021-12-31,0,1,2,3,4,5
operat_current,8709450.0,9.766263e+06,1.240258e+07,1.575055e+07,2.000227e+07,2.540170e+07,2.667179e+07
operat_non_curr,34280458.0,4.513190e+07,5.731485e+07,7.278649e+07,9.243456e+07,1.173865e+08,1.232558e+08
operat_liability,2252210.0,2.738722e+06,3.478016e+06,4.416876e+06,5.609172e+06,7.123319e+06,7.479484e+06
financial_assets,3233668.0,3.541016e+06,4.496881e+06,5.710775e+06,7.252348e+06,9.210055e+06,9.670558e+06
financial_liability,1286922.0,1.852345e+06,2.352369e+06,2.987371e+06,3.793785e+06,4.817884e+06,5.058778e+06


In [128]:
net_liability = operate.iloc[4,0]-operate.iloc[3,0]

In [56]:
net_operate = operate.iloc[0]+operate.iloc[1]-operate.iloc[2]

In [59]:
# 计算未来5年和永续经营的资本支出
capex_forecast = net_operate - net_operate.shift(1) 
#capex_forecast = capex_forecast.drop
capex_forecast = np.array(capex_forecast)[1:7]

In [60]:
capex_forecast

array([11421738.70901342, 14079978.74143925, 17880744.50632572,
       22707493.38275038, 28837180.43984333,  6783241.6889686 ])

In [33]:
# 预测5年的折旧
c = np.array(valuation['depreciation'])
c = c[0:3]
depre = c[::-1]

In [34]:
# g = np.full(5, growth_rate_max)    # 用指定增长率构造一个5元数组
# growth_rates = np.append(growth_rates, growth_rate_continue)

for i in np.arange(3,9):    # 构造整数序列
    d = (depre[(i-3)] + depre[(i-2)] + depre[(i-1)]) / 3     # 前三年的平均折旧金额
    d = d * growth_rate
    depre = np.append(depre, d)

depre_forecast = depre[3:9]    # 截取最后6项

In [35]:
depre_forecast

array([ 705125.93834435,  780488.03037883,  881038.59761258,
       1001836.50015853, 1127438.15163719, 1274306.89788041])

In [63]:
forecast_net_cash = profit_forecast + depre_forecast - capex_forecast


In [113]:
forecast_net_cash
ca1 = forecast_net_cash[0:5]    # 只计算今后5年
ca2 = np.concatenate(([0], ca1))    # 当年的现金流要设为0

In [121]:
contin = forecast_net_cash[5] / (discount_rate - continue_rate)
ca = np.concatenate((ca2, [contin]))

In [122]:
netpv = np.npv(0.12, ca)
print(netpv)

array([        0.        ,  -6377752.12959429,  -7789393.00450093,
       -10002206.04701871, -12819243.8057197 , -16424520.57564973,
        90579257.24422917])

In [129]:
net_pre_value = np.npv(discount_rate, ca)
stock_value = (net_pre_value - net_liability) / total_shares

In [130]:
stock_value[0]

21.41224452281828